### Part 3 
#### Function to estimate Transition Parameters using MLE

In [3]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
    
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences

modified_train_list = load_modified_train_file()   
dev_in_list = load_files()
sentences = split_into_sentences(dev_in_list)

In [4]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    

def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = modifiedWordDict[word_tag_pair_list[0]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict

def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict

def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber

modifiedWordDict = get_word_dict()
wordTagPairDict = count_word_tag_pair_dict()
giantEmissionDict = store_estimate_emission_fix()
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
giantTransitionDict = store_estimate_transition()


In [12]:
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences
sentences = split_into_sentences(dev_in_list)

In [6]:
def pi(i,tag,sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    #base case:
    if i == 0 and tag == "START":
        return 1
    elif i == 0 and tag == "STOP":
        return 0
    else:
        piList = {}
        word_at_v = sentence[i]
        for v in all_tags:
            piValue = pi(i - 1, tag, sentence) * giantTransitionDict[v + " " + u] * giantEmissionDict[word_at_v + " " + v]
            piList[v] = piValue
        max_node = max(piList, key=piList.get)
        return max_node

        
        

In [14]:
def viterbi(sentence):
    opti_path = []
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral", "START", "STOP"]
    for i in range(len(sentence)):
        opti_y_dict = {}
        for tag in all_tags:
            opti_y_at_i = pi(i, tag, sentence)
            opti_y_dict[tag] = opti_y_at_i
        max_i_path = max(opti_y_dict, key=opti_y_dict.get)
        opti_path.append(max_i_path)
    return opti_path

    
for i in range(1,len(sentences)):
    if i == 0:
        singleString = viterbi(sentences[i])
        print(singleString)